# PoC #3: Intelligent FAQ Chatbot 🤖

## Business Use Case
Automated customer support chatbot that answers frequently asked questions 24/7, reducing support tickets by 60-80%.

## Production Applications
- Customer support automation
- Employee onboarding assistant
- Product information helper
- Technical documentation Q&A

---

In [ ]:
!pip install -qU "ibm-watsonx-ai>=1.1.22"
print("✅ Dependencies installed")

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

WATSONX_APIKEY = getpass.getpass("IBM Cloud API Key: ")
PROJECT_ID = getpass.getpass("Watsonx Project ID: ")

model = Model(
    model_id="ibm/granite-3-8b-instruct",
    credentials=Credentials(url="https://us-south.ml.cloud.ibm.com", api_key=WATSONX_APIKEY),
    project_id=PROJECT_ID,
    params={
        GenParams.MAX_NEW_TOKENS: 200,
        GenParams.TEMPERATURE: 0.3,
        GenParams.DECODING_METHOD: DecodingMethods.GREEDY
    }
)
print("✅ Chatbot initialized")

## Knowledge Base (FAQ Database)

In [ ]:
# Production: Load from database/CMS
knowledge_base = {
    "shipping": {
        "question": "What are your shipping options and costs?",
        "answer": "We offer free standard shipping (5-7 business days) on orders over $50. Express shipping (2-3 days) costs $15, and overnight shipping is available for $30."
    },
    "returns": {
        "question": "What is your return policy?",
        "answer": "You can return items within 30 days for a full refund. Products must be unused and in original packaging. Return shipping is free for defective items."
    },
    "payment": {
        "question": "What payment methods do you accept?",
        "answer": "We accept all major credit cards (Visa, Mastercard, Amex), PayPal, Apple Pay, and Google Pay. We also offer buy-now-pay-later through Klarna."
    },
    "warranty": {
        "question": "Do your products come with a warranty?",
        "answer": "All products include a 1-year manufacturer warranty covering defects. Extended warranties (2-3 years) are available for purchase at checkout."
    },
    "international": {
        "question": "Do you ship internationally?",
        "answer": "Yes, we ship to over 50 countries. International shipping costs vary by destination and are calculated at checkout. Delivery typically takes 7-14 business days."
    },
    "track_order": {
        "question": "How can I track my order?",
        "answer": "You'll receive a tracking number via email once your order ships. You can also track orders by logging into your account and visiting the 'My Orders' section."
    },
}

print(f"📚 Knowledge base loaded: {len(knowledge_base)} topics")

## Chatbot Function

In [ ]:
def chat(user_question: str) -> str:
    """Process user question and generate response."""
    
    # Build context from knowledge base
    context = "\n\n".join([
        f"Q: {item['question']}\nA: {item['answer']}"
        for item in knowledge_base.values()
    ])
    
    prompt = f"""You are a helpful customer support chatbot. Answer the user's question based on the FAQ below.

FAQs:
{context}

User Question: {user_question}

Provide a friendly, concise answer. If the question isn't covered in the FAQs, politely say so and offer to connect them with a human agent.

Answer:"""
    
    try:
        response = model.generate_text(prompt=prompt)
        return response.strip()
    except Exception as e:
        return f"I apologize, but I'm experiencing technical difficulties. Please try again or contact support@company.com. Error: {str(e)}"

print("✅ Chatbot ready!")

## Demo: Common Customer Questions

In [ ]:
test_questions = [
    "How much does shipping cost?",
    "Can I return a product if I don't like it?",
    "Do you accept PayPal?",
    "Is there a warranty on your products?",
    "What's the weather like today?"  # Out of scope question
]

print("🤖 CHATBOT DEMO\n")
print("="*80)

for question in test_questions:
    print(f"\n👤 User: {question}")
    response = chat(question)
    print(f"🤖 Bot: {response}")
    print("-"*80)

## Interactive Chat Session

In [ ]:
print("\n💬 INTERACTIVE CHAT (type 'quit' to exit)\n")
print("="*80)

conversation_history = []

while True:
    user_input = input("\n👤 You: ").strip()
    
    if user_input.lower() in ['quit', 'exit', 'bye']:
        print("\n🤖 Bot: Thank you for chatting! Have a great day!")
        break
    
    if not user_input:
        continue
    
    response = chat(user_input)
    print(f"\n🤖 Bot: {response}")
    
    conversation_history.append({
        "user": user_input,
        "bot": response
    })

print(f"\n📊 Conversation turns: {len(conversation_history)}")

## Production Analytics

In [ ]:
# Analyze conversation patterns
if conversation_history:
    print("\n📈 CONVERSATION ANALYTICS")
    print("="*80)
    print(f"Total questions asked: {len(conversation_history)}")
    print(f"\nSample interactions:")
    for i, turn in enumerate(conversation_history[:3], 1):
        print(f"\n{i}. Q: {turn['user'][:60]}...")
        print(f"   A: {turn['bot'][:60]}...")
    print("="*80)

---

## Summary

### ✅ Features Demonstrated
- Context-aware responses from knowledge base
- Out-of-scope question handling
- Interactive conversation flow
- Conversation tracking

### 🚀 Production Enhancements
1. **Add RAG**: Integrate vector database for semantic search
2. **Multi-turn**: Add conversation memory/context
3. **Escalation**: Auto-route complex questions to humans
4. **Analytics**: Track resolution rates and user satisfaction
5. **Multi-language**: Support internationalization

### 💡 ROI Benefits
- ⏱️ 24/7 availability
- 📉 60-80% reduction in support tickets
- ⚡ Instant response times
- 💰 Lower customer support costs

---